# multiprocessing

## 11/30/2021
<a href="?print-pdf">print view</a>

In [1]:
%%html
<script src="http://bits.csb.pitt.edu/asker.js/lib/asker.js"></script>
<script>

require(['https://cdnjs.cloudflare.com/ajax/libs/Chart.js/2.2.2/Chart.js'], function(Ch){
 Chart = Ch;
});

$('head').append('<link rel="stylesheet" href="http://bits.csb.pitt.edu/asker.js/themes/asker.default.css" />');


//the callback is provided a canvas object and data 
var chartmaker = function(canvas, labels, data) {
  var ctx = $(canvas).get(0).getContext("2d");
     var dataset = {labels: labels,                     
    datasets:[{
     data: data,
     backgroundColor: "rgba(150,64,150,0.5)",
         fillColor: "rgba(150,64,150,0.8)",    
  }]};
  var myBarChart = new Chart(ctx,{type:'bar',data:dataset,options:{legend: {display:false},
        scales: {
            yAxes: [{
                ticks: {
                    min: 0,
                }
            }]}}});
};

$(".jp-InputArea .o:contains(html)").closest('.jp-InputArea').hide();



</script>

# OMETs

Please fill out.

https://teaching.pitt.edu/omet/student-information/

# Parallel Programming

There are several parallel programming models enabled by a variety of hardware (**multicore**, cloud computing, supercomputers, GPU).

<img src='http://images.anandtech.com/reviews/cpu/intel/SNBE/Core_I7_LGA_2011_Die.jpg' width="400">

# Threads vs. Processes

A **thread** of execution is the smallest sequence of programmed instructions that can be managed independently by an operating system scheduler. 

A **process** is an instance of a computer program.

<center><img src='http://upload.wikimedia.org/wikipedia/commons/a/a5/Multithreaded_process.svg' width="400"></center>

# Address Spaces

A process has its own *address space*.  An address space is a mapping of **virtual memory addresses** to **physical memory addresses** managed by the operating system.

Address spaces prevent processes from crashing other applications or the operating system - they can only access their *own* memory.

<center><img src='http://upload.wikimedia.org/wikipedia/commons/thumb/3/32/Virtual_address_space_and_physical_address_space_relationship.svg/587px-Virtual_address_space_and_physical_address_space_relationship.svg.png' width="350"></center>

# Threads vs. Processes

In [ ]:
import threading,time

cnt = [0]

def incrementCnt(cnt):
    for i in range(1000000): # a million times
        cnt[0] += 1

t1 = threading.Thread(target=incrementCnt,args=(cnt,))
t2 = threading.Thread(target=incrementCnt,args=(cnt,))

t1.start()
t2.start()

print(cnt[0])

What do we expect to print out?

In [3]:
%%html
<div id="thread1" style="width: 500px"></div>
<script>
$('head').append('<link rel="stylesheet" href="http://bits.csb.pitt.edu/asker.js/themes/asker.default.css" />');

    var divid = '#thread1';
	jQuery(divid).asker({
	    id: divid,
	    question: "What will print out?",
		answers: ['0','2','1000000','2000000',"I don't know"],
        server: "http://bits.csb.pitt.edu/asker.js/example/asker.cgi",
		charter: chartmaker})
    
$(".jp-InputArea .o:contains(html)").closest('.jp-InputArea').hide();


</script>

# The Answer

In [4]:
cnt = [0]

t1 = threading.Thread(target=incrementCnt,args=(cnt,))
t2 = threading.Thread(target=incrementCnt,args=(cnt,))

t1.start()
t2.start()

print(cnt) 
time.sleep(1)
print(cnt)
time.sleep(1)
print(cnt)

[71785]
[1273221]
[1273221]


# Threads vs. Processes

In [14]:
import multiprocess,time

def incrementCnt(cnt):
    for i in range(1000000): # a million times
        cnt[0] += 1
        
cnt = [0]

p1 = multiprocess.Process(target=incrementCnt,args=(cnt,))
p2 = multiprocess.Process(target=incrementCnt,args=(cnt,))

p1.start()
p2.start()

#what do we expect when we print out cnt[0]?

In [15]:
%%html
<div id="proc1" style="width: 500px"></div>
<script>
$('head').append('<link rel="stylesheet" href="http://bits.csb.pitt.edu/asker.js/themes/asker.default.css" />');

    var divid = '#proc1';
	jQuery(divid).asker({
	    id: divid,
	    question: "What will print out?",
		answers: ['0','2','1000000','2000000',"I don't know"],
        server: "http://bits.csb.pitt.edu/asker.js/example/asker.cgi",
		charter: chartmaker})
    
$(".jp-InputArea .o:contains(html)").closest('.jp-InputArea').hide();


</script>

# The Answer

In [16]:
cnt = [0]
p1 = multiprocess.Process(target=incrementCnt,args=(cnt,))
p2 = multiprocess.Process(target=incrementCnt,args=(cnt,))

p1.start()
p2.start()

print(cnt[0])
time.sleep(3)
print(cnt[0])

0
0


# Threads/Process Objects

Thread and Process Objects have the same interface 
 * `start` - start running the `target` function with (optional) `args`
 * `join` - wait for thread to finish before doing anything else
 * `is_alive` - returns true if thread is still running

# Example

In [17]:
t1 = threading.Thread(target=incrementCnt,args=(cnt,))
t2 = threading.Thread(target=incrementCnt,args=(cnt,))

t1.start() #start launches the thread to run target with args
t2.start()

print(cnt[0],t1.is_alive(),t2.is_alive())

t1.join() #join waits for a thread to finish
t2.join() #if you don't join a Process, it will become a zombie

print(cnt[0],t1.is_alive(),t2.is_alive())

196483 True True
1419271 False False


# Synchronization

When two threads update the same resource, their access to that resource needs to be gated.

There are a variety of synchronization primitives provided for both threads and processes (although they usually aren't needed for processes):

### Lock
Can be acquired by exactly one thread (calling acquire twice from the same thread will hang).  Must be released to be acquired by another thread.  Basically, just wrap the *critical section* with an acquire-release pair.

### RLock
A *recursive* lock. This is a lock that can be acquired multiple times by the same thread (and then must be released exactly the same number times).  This is especially useful for modular programming.  For example, you can acquire/release a lock around a function call without working about what that function is doing with the lock.


# Lock Example

In [18]:
lock = threading.Lock()

def incrementCnt(cnt):
    for i in range(1000000): # a million times
        lock.acquire()  #only one thread can acquire the lock at a time
        cnt[0] += 1 #this is the CRITICAL SECTION
        lock.release()

cnt = [0]
t1 = threading.Thread(target=incrementCnt,args=(cnt,))
t2 = threading.Thread(target=incrementCnt,args=(cnt,))

t1.start() #start launches the thread to run target with args
t2.start()

t1.join() 
t2.join() 

print(cnt[0])

2000000


# Communication

Threads communicate through their shared address space, and use locks to protect sensitive state.  Several classes provide a simplified interface for communication (these are available with processes as well, but the underlying implementation is different). 

### Queue
`Queue.Queue` (`multiprocess.Queue` for processes) provides a simple, thread-safe, first-in-first-out queue.

* `put`: put an element on the queue. **This will block if the queue has filled up**
* `get`: get an element from the queue. **This will block if the queue is empty**.  


# Communication

### Pipe
A pipe is a communication channel between processes that can send and receive messages.   *Unlike a queue, it is not okay for multiple threads to write to the same end of the pipe at the same time*.

`Pipe()` return a duple of `Connection` objects representing the ends of the pipe.  Each connection object has the following methods:

* `send`: sends data to other end of the pipe
* `recv`: waits for data from other end of the pipe (unless pipe closed, then `EOFError`)
* `close`: close the pipe

# Communication

Since they don't have a shared address space, it is recommended the *processes* use exclusively either `multiprocess.Queue` or `multiprocess.Pipe` to communicate.

Use a pipe for communication between two processes (server-client architecture).

Use a queue for one-way communication between many threads (producer-consumer architecture).

# Pipes

In [25]:
import multiprocess

def chatty(conn): #this takes a Connection object representing one end of a pipe
    msg = conn.recv()
    conn.send("you sent me "+msg)
    
(c1,c2) = multiprocess.Pipe()

p1 = multiprocess.Process(target=chatty,args=(c2,))
p1.start()

c1.send("Hello!")
result = c1.recv()
p1.join()

print(result)

you sent me Hello!


# Pools

`multiprocess` supports the concept of a *pool* of workers.  You initialize with the number of processes you want to run in parallel (the default is the number of CPUs on your system) and they are available for doing parallel work:

* `map`: the most important function - just like the built-in map, this will map a function to an iterable and return the result, but the *mapping will be done in parallel*.  Blocks until the full result is computed and the result is in the proper order.
* `map_async`: map without blocking
* `imap`: parallel imap - returns iterable instead of list.  The `next()` method of the returned iterable will block if necessary.
* `imap_unordered`: same as imap but returns values in order they are computed
* `close`: close the pool to prevent additional jobs from being submitted
* `join`: must call `close` first; waits for all jobs to complete

# Pool Example

In [20]:
def f(x):
    return x*x

pool = multiprocess.Pool(processes=4)

print(pool.map(f,range(20)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361]


# Producer/Consumer

In [21]:
def dowork(inQ, outQ):
    val = inQ.get()
    outQ.put(val*val)

inQ = multiprocess.Queue()
outQ = multiprocess.Queue()
pool = multiprocess.Pool(4, dowork, (inQ, outQ))

In [22]:
inQ.put(4)

In [23]:
outQ.get()

16

# Bad Example

In [24]:
#What is wrong with this code?
for i in range(10):
    inQ.put(i)

while not outQ.empty():
    print(outQ.get())

You should not produce and consume in the same thread.  If `outQ` fills up, `inQ` will fill up and the `put` will block.

The `empty`/`full` methods of the Queue class are pretty much **useless** since their result depends on when they are called.  Here, no values had been generated when it was called so nothing is printed.

In order communicate that there are no more values, you must send a *sentinal* value.

# Threads or Processes?

Normally, the choice between threads and processes depends on how data will be accessed and the level of communication between parallel tasks etc.

However, in Python, the answer is almost always **use multiprocessing, not threading**.

Why? The CPython interpretter has a *Global Interpretter Lock*.  This means that only **one** thread of python code is actually executed at any given time when using threads.  With processes, each process has its own interpretter (with its own lock).

<center><img width="300" src='../files/threadsprocesses.png'></center>

# Embarassingly Parallel

Writing correct, high performance parallel code can be difficult, but some in some cases it's trivial.

A problem is *embarassingly parallel* if it can be easily separated into *independent* subtasks (i.e., no need to communicate between threads) each of which does a substantial amount of computation.

Fortunately this is often the case.
 * Apply this filter to 1000 images
 * Process these 5000 protein structures
 * Compute RMSDs of all frames in a trajectory
 
In cases like these, using **Pools** will get you a significant speedup (by however many cores you have).

# Key Concepts

* You can running many things at the same time with threads/processes
* This is an easy way to make things faster, but can get complicated
* Use `multiprocess` not threads
* Processes can communicate with Queues/Pipes
* 90% of the time all you need are Pools, and they are not complicated

# Project

 * Take a search term as an argument
 * Use biopython to search the NCBI protein database for entries that match this term and are in the pdb
 * Extract the PDB entries
 * For each PDB entry, use ProDy to calculate the ANM modes (this can be done in parallel)
 * Sort the results based on the highest fractional variance in the first mode
 * Print out the top ten PDBs with the fractional variance of their first three modes

In [28]:
from Bio import Entrez
import re
seqfile = 'searchresults.fasta'
Entrez.email = "dkoes@pitt.edu"
records = Entrez.read(Entrez.esearch(db='protein',term='covid AND pdb[filter]',retmax=1000))
result = Entrez.efetch(db='protein',id=records['IdList'],rettype='fasta',retmode='text').read()
set(re.findall(r'>pdb\|(\S+?)\|',result))

{'5R7Y',
 '5R7Z',
 '5R80',
 '5R81',
 '5R82',
 '5R83',
 '5R84',
 '6LU7',
 '6LXT',
 '6LZE',
 '6M03',
 '6M0K',
 '6M17',
 '6M18',
 '6M1D',
 '6M39',
 '6M3M',
 '6M56',
 '6M71',
 '6VGY',
 '6VGZ',
 '6VH0',
 '6VH1',
 '6VH2',
 '6VH3',
 '6VW1',
 '6W2A',
 '6W41',
 '6W4B',
 '6W63',
 '6W79',
 '6WCO',
 '6WEY',
 '6WPS',
 '6WPT',
 '6WQF',
 '6WS6',
 '6WTJ',
 '6WTK',
 '6WTM',
 '6WTT',
 '6WUU',
 '6WX4',
 '6WZO',
 '6WZQ',
 '6X6P',
 '6X79',
 '6XA4',
 '6XBG',
 '6XBH',
 '6XBI',
 '6XCA',
 '6XCM',
 '6XCN',
 '6XDC',
 '6XE1',
 '6XEY',
 '6XEZ',
 '6XFN',
 '6XHL',
 '6XHM',
 '6XHN',
 '6XHO',
 '6XKL',
 '6XKP',
 '6XKQ',
 '6XMK',
 '6Y2E',
 '6Y2F',
 '6Y2G',
 '6Y84',
 '6YB7',
 '6YOR',
 '6YWK',
 '6YWL',
 '6YWM',
 '6YYT',
 '6Z97',
 '6ZB4',
 '6ZB5',
 '6ZCT',
 '6ZCZ',
 '6ZDG',
 '6ZDH',
 '6ZER',
 '6ZFO',
 '6ZLW',
 '6ZMT',
 '6ZN5',
 '6ZON',
 '6ZP4',
 '6ZPE',
 '6ZXN',
 '7BQY',
 '7BTF',
 '7BUY',
 '7BV1',
 '7BV2',
 '7BW1',
 '7BW4',
 '7BWJ',
 '7BYR',
 '7BZ5',
 '7BZF',
 '7C01',
 '7C02',
 '7C2K',
 '7C2L',
 '7C2Q',
 '7C2Y',
 '7C8R',
 